In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

#[a,b,c]: a-> 0-fast bowler, 1-spinner
          b-> 0-left arm, 1-right arm
          c-> for spinner 0-> spinning to right, 1-> spinning to left
                  for pacers 0-> fast, 1-> fast medium, 2-> medium

In [ ]:
types_of_batters={"Left Handed Bat":0,
                  "Right Handed Bat":1,
                  "Right hand Bat":0,
                  "Left hand Bat":1}

#[a,b,c]: a-> 0-fast bowler, 1-spinner
#          b-> 0-left arm, 1-right arm
#         c-> for spinner 0-> spinning to right, 1-> spinning to left
#              for pacers 0-> fast, 1-> fast medium, 2-> medium


types_of_bowlers={'Right-arm legbreak':[1,1,1],
                  'Right-arm offbreak':[1,1,0], 
                  'Left-arm orthodox':[1,0,1],
                  'Right-arm fast-medium':[0,1,1],
                  'Left-arm fast-medium':[0,0,1],
                  'Right-arm fast':[0,1,0],
                  'Left-arm fast':[0,0,0],
                  'Right-arm medium':[0,1,2],
                  'Left-arm chinaman':[1,0,0],
                  'Right arm Fast medium':[0,1,1],
                  'Right arm Medium fast':[0,1,1],
                  'Left-arm medium':[0,0,2],
                  'Right arm Medium':[0,1,2], 
                  'Legbreak':[1,1,1],
                  'Right arm Offbreak':[1,1,0],
                  'Slow Left arm Orthodox':[1,0,1],
                  'Left arm Fast medium':[0,0,1],
                  'Slow Left arm Orthodox':[1,0,1],
                  }

In [ ]:
batters_path="D:/Cricket Prediction Project/Players/Batters.csv"
df_batter=pd.read_csv(batters_path)
batters=list(df_batter["Name"])
teams=list(df_batter["Team"])

In [ ]:
bowlers_path="D:/Cricket Prediction Project/Players/Bowlers.csv"
df_bowler=pd.read_csv(bowlers_path)
bowlers=list(df_bowler["Name"])
bowler_type=list(df_bowler["Bowling"])
bowling_type = dict()
for i in range(len(bowlers)):
    bowling_type[bowlers[i]]=bowler_type[i]

In [ ]:
all_rounders_path="D:/Cricket Prediction Project/Players/Allrounders.csv"
df_allrounders=pd.read_csv(all_rounders_path)
all_rounders=list(df_allrounders["Name"])
allrounder_team=list(df_allrounders["Team"])
bowler_type=list(df_allrounders["Bowling"])
for i in range(len(all_rounders)):
    bowling_type[all_rounders[i]]=bowler_type[i]
    batters.append(all_rounders[i])
    teams.append(allrounder_team[i])

In [ ]:
len(batters)

921

In [ ]:
attributes=["match_id",
                "venue",
                "opposition",
                "bowler",
                "bowler_type",
                "bowler_type_tuple",
                "got_him_out",
                "runs_scored",
                "balls_faced",
                "fours",
                "sixes"]
def set_batter_data(player_name,team,path,data):
    for filename in os.listdir(path):
        name_parts=str(filename).split()
        match_id=name_parts[-1].split(".")[0]
        inning=""
        file_path=os.path.join(path,filename)
        df_1=pd.read_csv(os.path.join(file_path,"1st_Innings.csv"))
        df_2=pd.read_csv(os.path.join(file_path,"2nd_Innings.csv"))
        meta_data=json.load(open(os.path.join(file_path,"meta_data.json")))
        #print(meta_data["teams"])
        if( not (team in meta_data["teams"])):
            continue
        df=pd.DataFrame()
        if(player_name in list(df_1["batter"])):
            df=pd.DataFrame(df_1)
        elif(player_name in list(df_2["batter"])):
            df=pd.DataFrame(df_2)
        else:
            continue
        #print('1')
        data_match={
            "match_id":[],
            "venue":[],
            "opposition":[],
            "bowler":[],
            "bowler_type":[],
            "bowler_type_tuple":[],
            "got_him_out":[],
            "runs_scored_before_getting_out":[],
            "runs_scored":[],
            "balls_faced":[],
            "fours":[],
            "sixes":[]
        }
        opposition=meta_data["teams"][1] if meta_data["teams"][0]==team else meta_data["teams"][0]
        df_player=pd.DataFrame(df.loc[df["batter"]==player_name])
        bowler_set=set(list(df_player["bowler"]))
        for i in bowler_set:
            if (i in bowlers) or (i in all_rounders):
                data_match["venue"].append(meta_data["venue"])
                data_match["opposition"].append(opposition)
                data_match["match_id"].append(match_id)
                data_match["bowler"].append(i)
                data_match["bowler_type"].append(bowling_type[i])
                try:
                    data_match["bowler_type_tuple"].append(str(types_of_bowlers[bowling_type[i]]))
                except:
                    data_match["bowler_type_tuple"].append("not known")
                df_player_bowler=df_player.loc[df_player['bowler']==i]
                wicket_taken=list(df.query('batter == @player_name and bowler == @i and wicket == 1')["wicket"])
                fl=0
                if len(wicket_taken)>0:
                    fl=1
                data_match["got_him_out"].append(fl)
                data_match["runs_scored"].append(sum(list(df_player_bowler["run_hit"])))
                data_match["balls_faced"].append(len(df_player_bowler))
                data_match["fours"].append(len(df_player_bowler.loc[df_player_bowler['run_hit']==4]))
                data_match["sixes"].append(len(df_player_bowler.loc[df_player_bowler['run_hit']==6]))
        #print(pd.DataFrame(data_match))
        for i in attributes: 
            data[i]=data[i]+data_match[i]
        #print(pd.DataFrame(data))

In [ ]:
save="D:/Cricket Prediction Project/ODI/PLAYERS"
def save_player_data(name,team):
    path="D:/Cricket Prediction Project/ODI/ODI_MEN"
    data={
            "match_id":[],
            "venue":[],
            "opposition":[],
            "bowler":[],
            "bowler_type":[],
            "bowler_type_tuple":[],
            "got_him_out":[],
            "runs_scored":[],
            "balls_faced":[],
            "fours":[],
            "sixes":[]
        }
    set_batter_data(name,team,path,data)
    df=pd.DataFrame(data)
    fname=name+".csv"
    df.to_csv(os.path.join(save,fname))

In [ ]:
all_teams=["India","Australia","England","New Zealand","Afghanistan","Bangladesh","Ireland","Zimbabwe","Pakistan","South Africa","Sri Lanka","United Arab Emirates","Honk Kong","West Indies","Namibia","Nepal","Oman","Papua New Guinea","Scotland"]
for i in range(len(batters)):
    if teams[i] in all_teams:
        save_player_data(batters[i], teams[i])